### Imports and Data Load/Prep

In [27]:
import datetime
import multiprocessing as mp
import os
import pathlib

import dask.dataframe as dd
import datashader as ds
import datashader.transfer_functions as tf
import geopandas as gpd
import joblib
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pyarrow as pa
from colorcet import fire
from dask.distributed import Client, LocalCluster
from identify_district import load_precincts
from shapely.geometry import Point
from shapely.strtree import STRtree


In [28]:
def get_district(latitude:pa.float64, longitude:pa.float64, snames:np.ndarray, tree:STRtree)->str:
    """Fast function for finding which district a point lies inside

    Args:
        latitude (pa.float64): floating point latitude value
        longitude (pa.float64): floating point longitude value
        snames (np.ndarray): array of sector names indexed the same as the tree
        tree (STRtree): tree of geometries for quick lookup

    Returns:
        str: csv of sector names a point falls into
    """
    p = Point(longitude,latitude)
    idx = tree.query(p, 'within')
    if len(idx):
        return snames[idx[0]]
    return ''

In [29]:
root_dir = pathlib.Path("../data/nypd")
try:
    # data from https://data.cityofnewyork.us/Public-Safety/NYPD-Sectors/eizi-ujye
    sectors = joblib.load(root_dir/"sectors.pkl")
except FileNotFoundError:
    # convert geometry column to shapely objects

    sectors = pd.read_csv(root_dir/"NYPD_Sectors_20240306.csv").convert_dtypes(dtype_backend='pyarrow')

    sectors['the_geom'] = gpd.GeoSeries.from_wkt(sectors['the_geom'])
    sectors = gpd.GeoDataFrame(sectors, geometry='the_geom')

    joblib.dump(sectors, root_dir/"sectors.pkl")

try:
    nypdf = pd.read_parquet(root_dir/"precincts")
    if len(nypdf) == 0:
        raise FileNotFoundError
    # nypdf = joblib.load("../data/nypd/nypdf_precinct.pkl")
except FileNotFoundError:
    print("File not found, making new file.")
    # Create cluster to speed up compute
    # cluster = LocalCluster(n_workers=4, threads_per_worker=8, processes=True)
    # client = Client(cluster)
    worker_count = int(0.9 * mp.cpu_count()) # leave a core or so for other use
    with LocalCluster(
        n_workers=worker_count,
        processes=True,
        threads_per_worker=1,
        memory_limit='1GB', # per worker memory limit
    ) as cluster, Client(cluster) as client:
        print("View progress here", client.dashboard_link)
        # Create snames list for use in labeling points
        snames:np.ndarray = sectors['SCT_TEXT'].values
        # Build a tree of geometry for faster queries
        tree:STRtree = STRtree(sectors.geometry)
        # load main 311 data
        nypdf = pd.read_parquet("../data/311/2018/processed/agency=NYPD")
        nypdf = nypdf[nypdf['created_date']>= datetime.date(2018, 1, 1)]
        # drop data without any lat/long
        nypdf = nypdf.dropna(how='any',subset=['latitude','longitude'])
        # convert to dask frame with 4 partitions
        nypd_dd: dd.DataFrame = dd.from_pandas(nypdf[['latitude','longitude']], npartitions=worker_count)
        # compute which district each complaint fall into
        sct = nypd_dd.apply(lambda x: get_district(x['latitude'], x['longitude'], snames, tree), axis=1, meta=('sector', str))
        sct.compute()
        # add computed values to dataframe
        nypdf['sector'] = sct
        nypdf['precinct'] = nypdf['sector'].str.slice(0,3).str.lstrip("0").str.rstrip(r"ABCDEFGHIJKL")

        # save
        nypdf.to_parquet(root_dir/"precincts", partition_cols=['precinct','sector'])



### Plot Heat Map 

In [30]:
# get a data frame of just lat and long in numpy datatype
nypdf_lat_long = nypdf[["latitude", "longitude"]].astype(np.float64)
# create a static heatmap pixel wise
cvs = ds.Canvas(plot_width=2000, plot_height=2000)
# doesnt accept pyarrow types
agg = cvs.points(nypdf_lat_long, y='latitude', x='longitude')
img = tf.shade(agg, cmap=fire)[::-1].to_pil()

# get image coords
coords_lat, coords_lon = agg.coords['latitude'].values, agg.coords['longitude'].values
coordinates = [[coords_lon[0], coords_lat[0]],
            [coords_lon[-1], coords_lat[0]],
            [coords_lon[-1], coords_lat[-1]],
            [coords_lon[0], coords_lat[-1]]]

# create mapbox
fig = px.scatter_mapbox(nypdf[:1], lat='latitude', lon='longitude', zoom=9, opacity=0)
fig.data[0]['lat'] = fig.data[0]['lon'] = []
# add the generated heatmap to the mapbox
fig.update_layout(mapbox_style="carto-darkmatter",
                mapbox_layers = [
                {
                    "sourcetype": "image",
                    "source": img,
                    "coordinates": coordinates
                }]
)
if not os.path.exists("../plots/nypd_311_map_density.html"):
    fig.write_html("../plots/nypd_311_map_density.html")

fig.show()

### Prepare Data for Animation Plotting

In [31]:
sectors.set_index('SCT_TEXT', inplace=True)
sectors = pd.merge(sectors, nypdf['sector'].value_counts(), how='left', left_on='SCT_TEXT', right_index=True)

In [32]:
# reshaping data for animation plot
nypdf['created_M']=nypdf['created_H'].dt.month
monthly_count = nypdf[['created_M', 'sector', 'created_date']].groupby(['created_M', 'sector'], observed=False).count()
monthly_count.columns = ["Complaint Volume"]
monthly_count = monthly_count.unstack('sector')
monthly_count.index = monthly_count.index.rename('Date')
monthly_count.drop(('Complaint Volume', ''), axis=1, inplace=True)
# daily_count = daily_count.stack('sector').unstack('Date')
monthly_count.reset_index(drop=False, inplace=True)
monthly_count.columns = monthly_count.columns.droplevel()
monthly_count['Month'] = monthly_count['']
monthly_count.drop('', axis=1,inplace=True)
monthly_count.set_index('Month', inplace=True)
monthly_count = monthly_count.T
monthly_count = monthly_count.stack().reset_index()
monthly_count = pd.merge(monthly_count, sectors['the_geom'], how='left', left_on='sector', right_index=True)
monthly_count = monthly_count.rename({0:'Complaint Volume', 'sector':'Sector', 'the_geom':'Geometry'}, axis='columns')

In [33]:
monthly_count

,Sector,Month,Complaint Volume,Geometry
0,001A,1,103,MULTIPOLYGON (((-74.00589175107575 40.71200715...
1,001A,2,121,MULTIPOLYGON (((-74.00589175107575 40.71200715...
2,001A,3,122,MULTIPOLYGON (((-74.00589175107575 40.71200715...
3,001A,4,102,MULTIPOLYGON (((-74.00589175107575 40.71200715...
4,001A,5,105,MULTIPOLYGON (((-74.00589175107575 40.71200715...
...,...,...,...,...
3619,094C,8,210,MULTIPOLYGON (((-73.95439555417077 40.73911477...
3620,094C,9,245,MULTIPOLYGON (((-73.95439555417077 40.73911477...
3621,094C,10,241,MULTIPOLYGON (((-73.95439555417077 40.73911477...
3622,094C,11,211,MULTIPOLYGON (((-73.95439555417077 40.73911477...


### Plot Monthly Complaint Volume by Sector by Borough

In [34]:
for boro in nypdf['borough'].unique():
    if boro == 'Unspecified':
        continue
    if os.path.exists(f"../plots/animations/{boro}_311.html"):
        continue
    boro_sectors = list(nypdf[nypdf['borough']==boro]['sector'].unique())
    boro_sectors = [b for b in boro_sectors if b]
    lat = nypdf[nypdf['borough']==boro]['latitude'].mean()
    lon = nypdf[nypdf['borough']==boro]['longitude'].mean()
    monthly_count1 = monthly_count[monthly_count['Sector'].isin(boro_sectors)]

    monthly_count_sectors = gpd.GeoDataFrame(monthly_count1, geometry=gpd.GeoSeries.from_wkt(monthly_count1['Geometry']))
    monthly_count_sectors = monthly_count_sectors.set_index('Sector')
    fig = px.choropleth_mapbox(
        data_frame=monthly_count_sectors,
        animation_frame='Month',
        # animation_group='Sector',
        geojson=monthly_count_sectors.geometry,
        locations=monthly_count_sectors.index,
        color='Complaint Volume',
        range_color=[0,round(monthly_count1['Complaint Volume'].max(),-2)+100],
        center=dict(lat=lat, lon=lon),
        mapbox_style="open-street-map",
        opacity=0.7,
        zoom=8.5,
    )
    fig.write_html(f"../plots/animations/{boro}_311.html")


### Plot Yearly Complaint Volume by Sector By Borough

In [35]:

if not os.path.exists("../plots/animations/choropleth_sectors_animated1.html"):
    monthly_count_sectors = monthly_count.set_index('Sector').drop('Geometry', axis='columns')
    fig = px.choropleth_mapbox(
        data_frame=monthly_count_sectors,
        animation_frame='Month',
        # animation_group='Sector',
        geojson=gpd.GeoSeries.from_wkt(monthly_count['Geometry']),
        locations=monthly_count_sectors.index,
        color='Complaint Volume',
        range_color=[0,round(monthly_count['Complaint Volume'].max(),-2)+100],
        center=dict(lat=40.73390865622287, lon=-73.9232548285547),
        mapbox_style="open-street-map",
        opacity=0.7,
        zoom=8.5,
    )
    fig.write_html("../plots/animations/choropleth_sectors_animated1.html")

In [10]:


if not os.path.exists("../plots/choropleth_sectors.html"):
    fig = px.choropleth_mapbox(sectors,
                        geojson=sectors.geometry,
                        locations=sectors.index,
                        color="count",
                        center=dict(lat=40.73390865622287, lon=-73.9232548285547),
                        mapbox_style="open-street-map",
                        opacity=0.7,
                        zoom=8.5,
                        labels={'SCT_TEXT':"Sector", "count":"Complaint Volume"}
                        )
    fig.write_html("../plots/choropleth_sectors.html")

    fig.show()

In [11]:
fig = px.density_mapbox(nypdf, lat='latitude', lon='longitude', radius=1.5,
                        center=dict(lat=40.73390865622287, lon=-73.9232548285547), zoom=9,
                        mapbox_style="open-street-map")
# fig.show()

In [12]:
# from mpl_toolkits.basemap import Basemap
#
# agency = q2017_df["agency"].unique()
# # Generate unique colors for each unique value
# num_unique_values = len(agency)
# colors = plt.cm.rainbow(np.linspace(0, 1, num_unique_values))

# # Map unique values to unique colors
# color_map = dict(zip(agency, colors))

# avg_lat = q2017_df["latitude"].mean()
# avg_lon = q2017_df["longitude"].mean()
# print(avg_lon, avg_lat)
# fig, ax = plt.subplots(1, 1, figsize=(12, 8))
# m = Basemap(projection='gnom', lat_0=avg_lat, lon_0=avg_lon,
#                 width=60_000, height=40_000, resolution="f", ax=ax)
# m.fillcontinents(color="#FFDDCC", lake_color='#DDEEFF')
# m.drawmapboundary(fill_color="#DDEEFF")
# m.drawcoastlines()

# for group, color in color_map.items():
#     group_data = q2017_df[q2017_df['agency'] == group]
#     m.scatter(
#         group_data["longitude"], group_data["latitude"],
#         latlon=True, color=color, label=group, alpha = 0.1,
#         s=0.5
#         # c=q2017_df["agency"]
#         )
# ax.legend(loc="upper right", ncol=1,bbox_to_anchor=[1.25,1], scatterpoints=4)

### Test Precinct

In [13]:
pct = load_precincts()
pct.set_index('precinct', inplace=True)

In [14]:
# pct['precinct'] = pct['precinct'].astype('int64').convert_dtypes(dtype_backend='pyarrow')

In [15]:
# reshaping data for animation plot
# get the monthly count of complaints for each nypd sector
nypdf['created_M']=nypdf['created_H'].dt.month
monthly_count = nypdf[['created_M', 'precinct', 'created_date']].groupby(['created_M', 'precinct'], observed=False).count()
monthly_count.columns = ["Complaint Volume"]
monthly_count = monthly_count.unstack('precinct')
monthly_count.drop(('Complaint Volume', ''), axis=1, inplace=True)
monthly_count.reset_index(drop=False, inplace=True)
monthly_count.columns = monthly_count.columns.droplevel()
monthly_count['Month'] = monthly_count['']
monthly_count.drop('', axis=1,inplace=True)
monthly_count.set_index('Month', inplace=True)
monthly_count = monthly_count.T
monthly_count = monthly_count.stack().reset_index()
monthly_count = pd.merge(monthly_count, pct.geometry, how='left', left_on='precinct', right_index=True)
monthly_count = monthly_count.rename({0:'Complaint Volume', 'precinct':'Precinct', 'geometry':'Geometry'}, axis='columns')

In [16]:
monthly_count

,Precinct,Month,Complaint Volume,Geometry
0,1,1,406,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
1,1,2,453,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
2,1,3,512,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
3,1,4,532,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
4,1,5,962,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
...,...,...,...,...
919,94,8,661,"MULTIPOLYGON (((-73.96623 40.71827, -73.96624 ..."
920,94,9,721,"MULTIPOLYGON (((-73.96623 40.71827, -73.96624 ..."
921,94,10,713,"MULTIPOLYGON (((-73.96623 40.71827, -73.96624 ..."
922,94,11,587,"MULTIPOLYGON (((-73.96623 40.71827, -73.96624 ..."


In [17]:
borough_pcts = {'MANHATTAN': (1, 39), 'BRONX': (40, 59), 'BROOKLYN': (60, 99), 'QUEENS': (100, 119), 'STATEN ISLAND': (120, 200)}

for boro in nypdf['borough'].unique():
    if boro == 'Unspecified':
        continue
    elif os.path.exists(f"../plots/animations/{boro}_pcts_311.html"):
        continue
    lo, hi = borough_pcts[boro]
    boro_sectors = [str(i) for i in range(lo, hi+1)]
    lat = nypdf[nypdf['borough']==boro]['latitude'].mean()
    lon = nypdf[nypdf['borough']==boro]['longitude'].mean()
    monthly_count1 = monthly_count[monthly_count['Precinct'].isin(boro_sectors)]

    monthly_count_sectors = gpd.GeoDataFrame(monthly_count1, geometry='Geometry')
    monthly_count_sectors = monthly_count_sectors.set_index('Precinct')
    fig = px.choropleth_mapbox(
        data_frame=monthly_count_sectors,
        animation_frame='Month',
        # animation_group='Sector',
        geojson=monthly_count_sectors.geometry,
        locations=monthly_count_sectors.index,
        color='Complaint Volume',
        range_color=[0,round(monthly_count1['Complaint Volume'].max(),-2)+100],
        center=dict(lat=lat, lon=lon),
        mapbox_style="open-street-map",
        opacity=0.7,
        zoom=8.5,
    )

    fig.write_html(f"../plots/animations/{boro}_pcts_311.html")

## Optimized Animation

In [24]:
monthly_count

,Precinct,Month,Complaint Volume,Geometry
0,1,1,406,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
1,1,2,453,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
2,1,3,512,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
3,1,4,532,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
4,1,5,962,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
...,...,...,...,...
919,94,8,661,"MULTIPOLYGON (((-73.96623 40.71827, -73.96624 ..."
920,94,9,721,"MULTIPOLYGON (((-73.96623 40.71827, -73.96624 ..."
921,94,10,713,"MULTIPOLYGON (((-73.96623 40.71827, -73.96624 ..."
922,94,11,587,"MULTIPOLYGON (((-73.96623 40.71827, -73.96624 ..."


In [ ]:


months = list(monthly_count['Month'].unique())
precincts = list(monthly_count['Precinct'].unique())
precincts
# make list of continents
# make figure
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

fig_dict["layout"]["title"] = { "title": "NYPD Precinct 311 Complaint Volume by Month"}
fig_dict["layout"]["hovermode"] = "closest"
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Year:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# make data
year = 1952
for continent in continents:
    single_month_data = dataset[dataset["year"] == year]
    single_month_data = single_month_data[
        single_month_data["continent"] == continent]

    data_dict = {
        "x": list(single_month_data["lifeExp"]),
        "y": list(single_month_data["gdpPercap"]),
        "mode": "markers",
        "text": list(single_month_data["country"]),
        "marker": {
            "sizemode": "area",
            "sizeref": 200000,
            "size": list(single_month_data["pop"])
        },
        "name": continent
    }
    fig_dict["data"].append(data_dict)

# make frames
for month in months:
    frame = {"data": [], "name": str(year)}
    for precinct in precincts:
        single_month_data = monthly_count[(monthly_count["Month"] == month)&(monthly_count["Precinct"] == precinct)]
        data_dict = {
            "x": list(single_month_data["lifeExp"]),
            "y": list(single_month_data["gdpPercap"]),
            "mode": "markers",
            "text": list(single_month_data["country"]),
            "marker": {
                "sizemode": "area",
                "sizeref": 200000,
                "size": list(single_month_data["pop"])
            },
            "name": continent
        }
        frame["data"].append(data_dict)

    fig_dict["frames"].append(frame)
    slider_step = {"args": [
        [year],
        {"frame": {"duration": 300, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
        "label": year,
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)


fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()